In [1]:
gen_prompt = """Напишите вежливое письмо-напоминание клиенту о предстоящей дате оплаты по счёту №4829 от 10 ноября 2025 года. Укажите, что оплата должна быть произведена до 25 ноября 2025 года, и поблагодарите за сотрудничество."""

class_prompt="""Классифицируйте следующий запрос пользователя по одной из категорий: `["Billing", "Tech support", "Sales"]`.
Текст запроса:  
Не могу войти в личный кабинет — система выдаёт ошибку "Неверный логин или пароль", хотя данные ввожу верные."""

extraction_pormpt="""Из приведённого описания товара извлеките следующие поля: название, производитель, объём памяти и тип интерфейса. Оформите результат в виде JSON. 
Описание:  
SSD накопитель Samsung 980 PRO 2 ТБ – высокоскоростной накопитель формата M.2 с интерфейсом PCIe 4.0 x4, разработанный для геймеров и профессионалов, работающих с тяжёлыми приложениями. Обеспечивает скорость чтения до 7000 МБ/с."""

In [11]:
default_params = {}
extra_params = {"temperature": 1, "max_tokens": 2500, "top_p": 1, "repeat_penalty": 1.5}

In [2]:
BASE_URL = "http://localhost:11434/v1/chat/completions"

In [4]:
%pip install requests


   ---------------------------------------- 0/5 [urllib3]
   -------- ------------------------------- 1/5 [idna]
   ---------------- ----------------------- 2/5 [charset_normalizer]
   -------------------------------- ------- 4/5 [requests]
   ---------------------------------------- 5/5 [requests]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import requests

response = requests.get("http://localhost:11434/api/tags")
print(response.json())

{'models': [{'name': 'deepseek-r1:8b', 'model': 'deepseek-r1:8b', 'modified_at': '2025-11-21T15:08:14.2110761+03:00', 'size': 5225376047, 'digest': '6995872bfe4c521a67b32da386cd21d5c6e819b6e0d62f79f64ec83be99f5763', 'details': {'parent_model': '', 'format': 'gguf', 'family': 'qwen3', 'families': ['qwen3'], 'parameter_size': '8.2B', 'quantization_level': 'Q4_K_M'}}, {'name': 'qwen3:4b', 'model': 'qwen3:4b', 'modified_at': '2025-11-21T15:05:41.6842567+03:00', 'size': 2497293931, 'digest': '359d7dd4bcdab3d86b87d73ac27966f4dbb9f5efdfcc75d34a8764a09474fae7', 'details': {'parent_model': '', 'format': 'gguf', 'family': 'qwen3', 'families': ['qwen3'], 'parameter_size': '4.0B', 'quantization_level': 'Q4_K_M'}}, {'name': 'gemma3:1b', 'model': 'gemma3:1b', 'modified_at': '2025-11-21T15:02:13.1913542+03:00', 'size': 815319791, 'digest': '8648f39daa8fbf5b18c7b4e6a8fb4990c692751d49917417b8842ca5758e7ffc', 'details': {'parent_model': '', 'format': 'gguf', 'family': 'gemma3', 'families': ['gemma3'], '

In [8]:
models = ['deepseek-r1:8b','qwen3:4b', 'gemma3:1b']

In [9]:
import time
import json
import requests
from datetime import datetime

def ask_model(model_name, user_prompt, extra_params):
    request_data = {
        "model": model_name,
        "messages": [{"role": "user", "content": user_prompt}],
    }
    request_data.update(extra_params)

    start_time = time.time()

    response = requests.post(BASE_URL, json=request_data, timeout=120)

    duration = time.time() - start_time

    try:
        result = response.json()
    except ValueError:
        result = {"error": response.text}

    return result, duration

In [12]:
prompts = {
    "generation": gen_prompt,
    "classification": class_prompt,
    "extraction": extraction_pormpt
}

mods = {
    "default": default_params,
    "precise": extra_params
}

with open("res.json", "w", encoding="utf-8") as output_file:
    for model in models:
        for prompt_id, prompt_text in prompts.items():
            for mode_name, mode_params in mods.items():
                answer, seconds = ask_model(model, prompt_text, mode_params)

                log_entry = {
                    "timestamp": datetime.now().isoformat(),
                    "model": model,
                    "prompt_id": prompt_id,
                    "prompt": prompt_text,
                    "mode": mode_name,
                    "params": mode_params,
                    "elapsed_s": seconds,
                    "response": answer
                }

                output_file.write(json.dumps(log_entry, ensure_ascii=False) + "\n")
                print(f"Записано: {model} | {prompt_id} | {mode_name} | {seconds:.2f} с")

Записано: deepseek-r1:8b | generation | default | 7.90 с
Записано: deepseek-r1:8b | generation | precise | 9.80 с
Записано: deepseek-r1:8b | classification | default | 6.09 с
Записано: deepseek-r1:8b | classification | precise | 3.89 с
Записано: deepseek-r1:8b | extraction | default | 13.77 с
Записано: deepseek-r1:8b | extraction | precise | 6.37 с
Записано: qwen3:4b | generation | default | 20.74 с
Записано: qwen3:4b | generation | precise | 20.03 с
Записано: qwen3:4b | classification | default | 3.54 с
Записано: qwen3:4b | classification | precise | 3.43 с
Записано: qwen3:4b | extraction | default | 9.97 с
Записано: qwen3:4b | extraction | precise | 18.66 с
Записано: gemma3:1b | generation | default | 2.08 с
Записано: gemma3:1b | generation | precise | 1.17 с
Записано: gemma3:1b | classification | default | 0.26 с
Записано: gemma3:1b | classification | precise | 0.35 с
Записано: gemma3:1b | extraction | default | 0.39 с
Записано: gemma3:1b | extraction | precise | 0.38 с


In [13]:
import json

with open("res.json", "r", encoding="utf-8") as f:
    for line in f:
        record = json.loads(line)
        model = record["model"]
        content = record["response"]["choices"][0]["message"]["content"]
        print(f"Модель: {model}")
        print(content)
        print()

Модель: deepseek-r1:8b
Здравствуйте!

Намекаем, что у нас есть счет №4829 от 10 ноября 2025 года.

По его статьям намечено произвести оплату до 25 ноября 2025 года.

Вы, конечно, помните об этом предстоящем платеже.

Благодарим Вас за интерес к нашим услугам и за плодотворное содействие в наших общих делах.

Заранее благодарим за Ваше понимание и оперативное действие.

С уважением,

[Ваше название компании]

Модель: deepseek-r1:8b
Конечно, вот проект вежливого письма-напоминания:

---

Уважаемый [Имя клиента]!

Нам довелось вспомнить о предстоящей дате оплаты по вашему счету №4829 от 10 ноября 2025 года. Подтверждаю, что оплата по данному счету назначена на 25 ноября 2025 года.

Хочу также отметить, что мы высоко ценим комфортное и своевременное сотрудничество с вами. Благодарим вас за продолжение работы с нашей компанией.

Просьба оплатить в удобном вам способе до указанной даты. Все детали оплаты предоставлены ранее, так же информацию можно уточнить по контактам.

Очень признателны з

In [16]:
import json
from collections import defaultdict

records = []
with open("res.json", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if line:
            records.append(json.loads(line))

grouped = defaultdict(list)
for r in records:
    key = (r["model"], r["prompt_id"])
    grouped[key].append(r)

for (model, prompt_id), runs in grouped.items():
    print(f"Model: {model}")
    print(f"Prompt ID: {prompt_id}")

    runs_sorted = sorted(runs, key=lambda x: x["mode"])
    for r in runs_sorted:
        mode = r["mode"]
        params = r["params"]
        elapsed = r["elapsed_s"]
        content = r["response"]["choices"][0]["message"]["content"]
        usage = r["response"].get("usage", {})
        prompt_tokens = usage.get("prompt_tokens", "N/A")
        completion_tokens = usage.get("completion_tokens", "N/A")
        total_tokens = usage.get("total_tokens", "N/A")

        gen_speed = "N/A"
        if isinstance(completion_tokens, int) and elapsed > 0:
            gen_speed = round(completion_tokens / elapsed, 1)

        print(f"Mode: {mode}")
        print(f"Params: {params}")
        print(f"Elapsed: {elapsed:.2f} s")
        print(f"Prompt tokens: {prompt_tokens}")
        print(f"Completion tokens: {completion_tokens}")
        print(f"Total tokens: {total_tokens}")
        print(f"Generation speed: {gen_speed} tokens/s")
        print()

Model: deepseek-r1:8b
Prompt ID: generation
Mode: default
Params: {}
Elapsed: 7.90 s
Prompt tokens: 100
Completion tokens: 497
Total tokens: 597
Generation speed: 62.9 tokens/s

Mode: precise
Params: {'temperature': 1, 'max_tokens': 2500, 'top_p': 1, 'repeat_penalty': 1.5}
Elapsed: 9.80 s
Prompt tokens: 100
Completion tokens: 747
Total tokens: 847
Generation speed: 76.2 tokens/s

Model: deepseek-r1:8b
Prompt ID: classification
Mode: default
Params: {}
Elapsed: 6.09 s
Prompt tokens: 80
Completion tokens: 469
Total tokens: 549
Generation speed: 77.0 tokens/s

Mode: precise
Params: {'temperature': 1, 'max_tokens': 2500, 'top_p': 1, 'repeat_penalty': 1.5}
Elapsed: 3.89 s
Prompt tokens: 80
Completion tokens: 300
Total tokens: 380
Generation speed: 77.1 tokens/s

Model: deepseek-r1:8b
Prompt ID: extraction
Mode: default
Params: {}
Elapsed: 13.77 s
Prompt tokens: 142
Completion tokens: 1058
Total tokens: 1200
Generation speed: 76.9 tokens/s

Mode: precise
Params: {'temperature': 1, 'max_token